In [1]:
import time
import datetime
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import geopandas as gpd

In [14]:
now = datetime.datetime.now()
now.isoformat()

'2022-04-20T11:54:58.909464'

In [3]:
zaryadye_park = '55.749435, 37.629416'

In [4]:
grid = gpd.read_file('../data/shp/moscow_grid.shp')
grid.head()

,public,car,walk,cycle,date,geometry
0,None,None,None,None,None,POINT (37.61878 55.48147)
1,None,None,None,None,None,POINT (37.62668 55.48156)
2,None,None,None,None,None,POINT (37.56325 55.48533)
3,None,None,None,None,None,POINT (37.57116 55.48542)
4,None,None,None,None,None,POINT (37.61071 55.48587)


In [5]:
# configure Yandex Maps specific class names
route_button = 'route-control__inner'

modes = {
    # mode: (button_class, result_class)
    'car': ('_mode_auto', 'auto-route-snippet-view__route-title-primary'),
    'public': ('_mode_masstransit', 'masstransit-route-snippet-view__route-duration'),
    'walk': ('_mode_pedestrian', 'pedestrian-route-snippet-view__route-title-primary'),
    'cycle': ('_mode_bicycle', 'bicycle-route-snippet-view__route-title-primary')
}

In [6]:
for mode, (btn, res) in modes.items():
    print(mode, btn, res)

car _mode_auto auto-route-snippet-view__route-title-primary
public _mode_masstransit masstransit-route-snippet-view__route-duration
walk _mode_pedestrian pedestrian-route-snippet-view__route-title-primary
cycle _mode_bicycle bicycle-route-snippet-view__route-title-primary


In [7]:
def look_up_transit_time(from_, to, driver):

    driver.get("https://yandex.ru/maps")

    # Let the page load: wait till DOM is fully constructed
    time.sleep(2)

    # Enter the route construction mode
    driver.find_element(by=By.CLASS_NAME, value=route_button).click()

    # Construct the keystroke sequence
    actions = ActionChains(driver)
    actions.send_keys(from_)
    actions.send_keys(Keys.ENTER)
    actions.send_keys(Keys.TAB)
    actions.send_keys(to)
    actions.send_keys(Keys.ENTER)

    # Type everything in
    time.sleep(1)
    actions.perform()
    
    time.sleep(1)
    times = dict()
    for mode, (btn, res) in modes.items():
        # Activate the corresponding mode by clicking the button
        driver.find_element(by=By.CLASS_NAME, value=btn).click()
        time.sleep(1.5)
        try:
            travel_time = driver.find_element(by=By.CLASS_NAME, value=res).text
        except selenium.common.exceptions.NoSuchElementException:
            travel_time = 'no route'
        
        times[mode] = travel_time

    return times

In [8]:
geometry = grid.loc[0, 'geometry']
point = f'{geometry.y}, {geometry.x}'

with webdriver.Firefox() as driver:
    ttt = look_up_transit_time(point, zaryadye_park, driver)

In [9]:
for mode, time_ in ttt.items():
    print(mode, time_)

car 1 ч 17 мин
public 1 ч 36 мин
walk 7 ч 9 мин
cycle 3 ч 19 мин


In [17]:
with webdriver.Firefox() as driver:
    
    # iterate over the grid to find best ime for each point
    for i, row in grid.iterrows():
        
        now = datetime.datetime.now()
        if now.hour < 8 or now.hour > 19:
            raise ValueError("It's either too early or too late! Results might be skeewed.")
        
        geometry = row['geometry']
        point = f'{geometry.y:.6f}, {geometry.x:.6f}'
        times = look_up_transit_time(point, zaryadye_park, driver)
        
        for mode, time_ in times.items():
            grid.loc[i, mode] = time_.replace('ч', 'h').replace('мин', 'm')

        grid.loc[i, 'date'] = now.isoformat()
                
        # Be nice to the Yandex Maps server and separate the requests a little
        time.sleep(3)
        
        if i > 0:
            break

In [18]:
grid.head()

,public,car,walk,cycle,date,geometry
0,1 h 36 m,1 h 14 m,7 h 9 m,3 h 19 m,2022-04-20T11:56:58.032570,POINT (37.61878 55.48147)
1,1 h 42 m,1 h 17 m,7 h 10 m,3 h 24 m,2022-04-20T11:57:14.648913,POINT (37.62668 55.48156)
2,None,None,None,None,None,POINT (37.56325 55.48533)
3,None,None,None,None,None,POINT (37.57116 55.48542)
4,None,None,None,None,None,POINT (37.61071 55.48587)
